# Semantic SfM of single scene in Scannet

### 1. Download entire Scannet dataset

Find information on ScanNet website: http://www.scan-net.org/ScanNet/

After download the data, you should extract data from .sens using [SensReader/reader.py](https://github.com/ScanNet/ScanNet/tree/master/SensReader/python). All the extracted data should be stored in `output` folder under a scene folder, e.g., `scannet/scans/scene0000_00/output`.

```bash
python3 reader.py --filename /home/zchen256/semantic_SfM/data/scannet/scans/scene0000_00/scene0000_00.sens --output_path /home/zchen256/semantic_SfM/data/scannet/scans/scene0000_00/output --export_depth_images --export_color_images --export_poses --export_intrinsics
```

### 2. Extract Scannet data to prepare SSfM data

In [1]:
from scene_extractor import SceneExtractor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
scene_dir = '../../data/scannet/scans/scene0000_00'
save_dir = '../../data/scene0000_00'

In [3]:
scene_extractor = SceneExtractor(scene_dir, save_dir)
scene_extractor.extract_photos()
scene_extractor.extract_depths()
scene_extractor.extract_segmentations()
scene_extractor.extract_reconstruction()
scene_extractor.extract_ground_truth()

Extracted 5578 segmentations


### 3. Select keyimages based on blur

In [7]:
from select_keyimages import select_scannet_keyimages

In [ ]:
scene_dir = save_dir
select_scannet_keyimages(scene_dir, ratio=0.2, threshold=180, file_cluster_size=30, n_jobs=8)

### 4. Create projection associations

In [2]:
from ssfm.probabilistic_projection import *
import os
import yaml

In [3]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.005)
pointcloud_projector.read_scannet_camera_parameters('../../data/scene0000_00')
pointcloud_projector.read_scannet_mesh('../../data/scene0000_00/reconstructions/mesh_vertices_color.npy')

In [4]:
# batch project
image_folder_path = '../../data/scene0000_00/photos'
save_folder_path = '../../data/scene0000_00/associations'

keyimages_path = os.path.join(save_folder_path, 'keyimages.yaml')

assert os.path.exists(keyimages_path), 'Keyimages not found'

with open(keyimages_path, 'r') as f:
    keyimages = yaml.safe_load(f)

# replace .npy with .jpg in keyimages, keyimages is a list of strings
image_list = [os.path.splitext(image)[0] + '.jpg' for image in keyimages]
# sort the image_list based on the values in the image names
image_list.sort(key=lambda x: int(x.split('/')[-1].split('_')[-1].split('.')[0]))

In [7]:
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, save_folder_path, num_workers=16, save_depth=False)

Processing frames: 100%|██████████| 614/614 [02:28<00:00,  4.13it/s]


In [5]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [8]:
smc_solver = KeyimageAssociationsBuilder(image_list, '../../data/scene0000_00/associations', '../../data/scene0000_00/segmentations_gt')
smc_solver.build_associations()

100%|██████████| 614/614 [00:02<00:00, 205.20it/s]


In [6]:
smc_solver.find_min_cover()

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 5585       | 6.86                 |
| Number of points covered by less than or equal to 1 image    | 11823      | 14.53                |
| Number of points covered by less than or equal to 3 images   | 22172      | 27.25                |
| Number of points covered by less than or equal to 5 images   | 29935      | 36.79                |


### 5. Estimate memory usage

In [6]:
from ssfm.memory_calculator import memory_calculator

In [14]:
# pointcloud file
npy_file = "../../data/scene0000_00/reconstructions/mesh_vertices_color.npy"
# image file sample
image_file = "../../data/scene0000_00/photos/0.jpg"
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(npy_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+-----------------------+
|              Memory Type               |  Memory Required (GB) |
+----------------------------------------+-----------------------+
|      Segmentation for each image       |  0.002336740493774414 |
| Pixel2point association for each image |  0.004673480987548828 |
| Point2pixel association for each image | 0.0003031231462955475 |
|                                        |                       |
|      Segmentation for all images       |   0.6402668952941895  |
| Pixel2point association for all images |   1.280533790588379   |
| Point2pixel association for all images |  0.08305574208498001  |
|          pc_segmentation_ids           | 0.0015156157314777374 |
|         pc_segmentation_probs          | 0.0015156157314777374 |
|          keyimage_association          |  0.020763935521245003 |
|                 Total                  |   2.027651594951749   |
+----------------------------------------+--------------------

### 6. Run object registration

In [11]:
from ssfm.object_registration import *
from ssfm.post_processing import *


In [9]:
pointcloud_path = '../../data/scene0000_00/reconstructions/mesh_vertices_color.npy'
segmentation_folder_path = '../../data/scene0000_00/segmentations_gt'
image_folder_path = '../../data/scene0000_00/photos'
association_folder_path = '../../data/scene0000_00/associations'

keyimage_associations_file_name = 'associations_keyimage.npy'

In [12]:
# Create object registration
obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path, keyimage_associations_file_name=keyimage_associations_file_name, image_list=image_list)

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)


# save semantic point cloud
obr.save_semantic_pointcloud('../../data/scene0000_00/semantic_model.las')

Processing images: 100%|██████████| 614/614 [04:16<00:00,  2.39it/s]


In [13]:
image_id = 613
semantics_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_path, save_las_path, remove_small_N=10, nearest_interpolation=10)

Before removing small semantics: 
maximum of semantics:  5847
number of unique semantics:  88
After removing small semantics: 
number of unique semantics:  65


In [14]:
semantic_pc_file_path = '../../data/scene0000_00/associations/semantics/semantics_{}.las'.format(image_id)
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../../data/scene0000_00/associations/semantics/semantics_{}_shuffled.las'.format(image_id)
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  64


In [15]:
from validation import *

In [16]:
scene_folder_path = '../../data/scene0000_00'
validator = ScannetValidation(scene_folder_path)
results = validator.validate(np.arange(0.5, 1.0, 0.05))
#results = validator.validate([0.5, 0.6])
mAP = results['AP']
mAR = results['AR']
print('mAP list: ', mAP, ' mAR list: ', mAR)
mAP = np.sum(mAP) / len(mAP)
mAR = np.sum(mAR) / len(mAR)
print('mAP: ', mAP, ' mAR: ', mAR)
    

Unique semantics in the ground truth:  68
Using the last prediction file:  ../../data/scene0000_00/associations/semantics/semantics_613_shuffled.las
Number of unique semantics in the prediction:  64
Number of unique semantics in the ground truth:  68
TP:  45  FP:  19  FN:  23
TP:  43  FP:  21  FN:  25
TP:  41  FP:  23  FN:  27
TP:  38  FP:  26  FN:  30
TP:  32  FP:  32  FN:  36
TP:  28  FP:  36  FN:  40
TP:  24  FP:  40  FN:  44
TP:  19  FP:  45  FN:  49
TP:  7  FP:  57  FN:  61
TP:  1  FP:  63  FN:  67
mAP list:  [0.703125, 0.671875, 0.640625, 0.59375, 0.5, 0.4375, 0.375, 0.296875, 0.109375, 0.015625]  mAR list:  [0.6617647058823529, 0.6323529411764706, 0.6029411764705882, 0.5588235294117647, 0.47058823529411764, 0.4117647058823529, 0.35294117647058826, 0.27941176470588236, 0.10294117647058823, 0.014705882352941176]
mAP:  0.434375  mAR:  0.4088235294117647
